# Installation

In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
Sat Apr 10 20:24:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |         

In [5]:
!pip install tensorflow
!pip install Keras

In [6]:
!python --version

Python 3.7.10


In [ ]:
import sys
sys.path.append('..')

import gc
import numpy as np
import cv2
import os.path
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model

import debugger
from deeplabv3 import Deeplabv3
from deep_neural_net import *
from predict_data_generator import DataGenerator
from UserParams import UserParams

In [ ]:
img_path = dataset_folder + dataset_name + img_folder
    
if constants.self_training_type is None:
    save_path = save_path + '{}/frame{}_{}_repeat{}/'.format(dataset_name, str(frame), model_name, str(repeat_index))
else:
    save_path = save_path + '{}_{}/frame{}_repeat{}/'.format(model_name, dataset_name, str(frame), str(repeat_index))
print('save_path:', save_path)
if os.path.isdir(save_path) == 0:
    os.makedirs(save_path)

# ------------------- Data loading -------------------
a_strategy = constants.strategy_type
if 'TIRF' in dataset_name and 'specialist' in constants.strategy_type:
    a_strategy = constants.strategy_type + '_normalize'

prediction_data_generator = DataGenerator(img_path, frame, 128, 68, a_strategy, img_format=constants.img_format)
imgs_val, namelist, image_cols, image_rows, orig_cols, orig_rows = prediction_data_generator.get_expanded_whole_frames()

print('img size:', image_rows, image_cols)
print('orig img size:', orig_rows, orig_cols)
print('imgs_val: ', imgs_val.dtype, imgs_val.shape)

# ------------------- Load trained Model -------------------

weights_path = constants.get_trained_weights_path(str(frame), model_name, str(repeat_index))

if "VGG19_dropout" in str(constants.strategy_type):
    model = VGG19_dropout(image_rows, image_cols, 0, image_cols-orig_cols, image_rows-orig_rows, weights_path=weights_path)
elif "unet" in str(constants.strategy_type):
    model = UNet(image_rows, image_cols, 0, image_cols-orig_cols, image_rows-orig_rows, weights_path=weights_path)

print('model layers: ', len(model.layers))
plot_model(model, to_file='model_plots/model_round{}_{}_predict.png'.format(constants.round_num, constants.strategy_type), show_shapes=True, show_layer_names=True, dpi=144)

# ------------------- predict segmented images and save them -------------------

if "feature_extractor" in str(constants.strategy_type):
    segmented_output, style_output = model.predict(imgs_val, batch_size = 1, verbose = 1)
    np.save(save_path + 'style_feature_vector.npy', style_output)
else:
    segmented_output = model.predict(imgs_val, batch_size = 1, verbose = 1)

segmented_output = 255 * segmented_output  # 0=black color and 255=white color

if "deeplabv3" == str(constants.strategy_type) or "EFF_B7" == str(constants.strategy_type) or "EFF_B7_no_preprocessing" == str(constants.strategy_type):
    # move last channel to first channel
    segmented_output = np.moveaxis(segmented_output, -1, 1)
    print(segmented_output.shape)

for f in range(len(namelist)):
    if constants.strategy_type == 'movie3' or constants.strategy_type == 'movie3_loss':
        out = segmented_output[f, 1, :, :]
    else:
        out = segmented_output[f, 0, :, :]
    cv2.imwrite(save_path + namelist[f], out)
K.clear_session()


if __name__ == "__main__":
    K.set_image_data_format('channels_first')
    constants = UserParams('predict')

    root_prediciton_path = "results/predict_wholeframe_round{}_{}/".format(constants.round_num, constants.strategy_type)

    if len(constants.model_names) != 1 and len(constants.dataset_names) != len(constants.model_names):
        raise Exception('Length of Dataset names and Model names are not the same')

    # for self training, ABCD model predicts for dataset A,B,C,D
    # for test set prediction, ABCD model predicts the dataset E
    for repeat_index in range(constants.REPEAT_MAX):
        for frame in constants.frame_list:
            for model_index in range(len(constants.model_names)): # len(constants.model_names)
                model_name = constants.model_names[model_index]
                dataset_folder = constants.dataset_folders[model_index]
                dataset_name = constants.dataset_names[model_index]
                img_folder = constants.img_folders[model_index]
                prediction(constants, model_name, dataset_folder, dataset_name, frame, repeat_index, img_folder, root_prediciton_path)
            gc.collect()